In [17]:
import torch

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")


c:\Users\adnan\Documents\Projects\xray-fracture-implementation\models


In [13]:
from torchvision.transforms import functional as F
from torchvision.datasets.vision import VisionDataset
from torchvision.datasets.utils import download_and_extract_archive, verify_str_arg
from torchvision.ops import box_area

from PIL import Image

import collections
import os
from xml.etree.ElementTree import Element as ET_Element

try:
    from defusedxml.ElementTree import parse as ET_parse
except ImportError:
    from xml.etree.ElementTree import parse as ET_parse
from typing import Any, Callable, Dict, List, Optional, Tuple


class FracAtlasDetection(VisionDataset):
    """FracAtlas dataset."""

    def __init__(
        self,
        root: str,
        image_set: str = "train",
        download: bool = False,
        transform: Optional[Callable] = None,
        target_transform: Optional[Callable] = None,
        transforms: Optional[Callable] = None,
    ):
        super().__init__(root, transforms, transform, target_transform)

        valid_image_sets = ["test", "train", "val"]
        self.image_set = verify_str_arg(image_set, "image_set", valid_image_sets)

        self.url = "https://figshare.com/ndownloader/files/41725659"
        self.filename = "FracAtlas.zip"

        if download:
            download_and_extract_archive(self.url, self.root, filename=self.filename)
        if not os.path.isdir(self.root):
            raise RuntimeError(
                "Dataset not found or corrupted. You can use download=True to download it"
            )

        splits_dir = os.path.join(self.root, "Utilities", "Fracture Split")
        splits_f = os.path.join(splits_dir, image_set.rstrip("\n") + ".csv")
        with open(os.path.join(splits_f)) as f:
            file_names = [
                os.path.splitext(x.strip())[0]
                for x in f.readlines()
                if x.strip() != "image_id"
            ]

        image_dir = os.path.join(self.root, "images")
        self.images = [os.path.join(image_dir, x + ".jpg") for x in file_names]

        target_dir = os.path.join(self.root, "Annotations", "PASCAL VOC")
        self.targets = [os.path.join(target_dir, x + ".xml") for x in file_names]

        assert len(self.images) == len(self.targets)

    @property
    def annotations(self) -> List[str]:
        return self.targets

    def __getitem__(self, index: int) -> Tuple[Any, Any]:
        """
        Args:
            index (int): Index

        Returns:
            tuple: (image, target) where target is a dictionary of the XML tree.
        """
        img = Image.open(self.images[index]).convert("RGB")
        img = F.to_tensor(img)
        voc_dict = self.parse_voc_xml(ET_parse(self.annotations[index]).getroot())
        target = self.voc_dict_to_target(index, voc_dict)
        # print(target)

        if self.transforms is not None:
            img, target = self.transforms(img, target)

        return img, target

    def __len__(self) -> int:
        return len(self.images)

    @staticmethod
    def parse_voc_xml(node: ET_Element) -> Dict[str, Any]:
        voc_dict: Dict[str, Any] = {}
        children = list(node)
        if children:
            def_dic: Dict[str, Any] = collections.defaultdict(list)
            for dc in map(FracAtlasDetection.parse_voc_xml, children):
                for ind, v in dc.items():
                    def_dic[ind].append(v)
            if node.tag == "annotation":
                def_dic["object"] = [def_dic["object"]]
            voc_dict = {
                node.tag: {
                    ind: v[0] if len(v) == 1 else v for ind, v in def_dic.items()
                }
            }
        if node.text:
            text = node.text.strip()
            if not children:
                voc_dict[node.tag] = text
        return voc_dict

    def voc_dict_to_target(self, index: int, item):
        width = int(item["annotation"]["size"]["width"])
        height = int(item["annotation"]["size"]["height"])

        num_fractures = len(item["annotation"]["object"])

        boxes = [
            (
                int(obj["bndbox"]["xmin"]),
                int(obj["bndbox"]["ymin"]),
                int(obj["bndbox"]["xmax"]),
                int(obj["bndbox"]["ymax"]),
            )
            for obj in item["annotation"]["object"]
        ]

        # guard against no boxes via resizing (https://github.com/pytorch/vision/blob/aa32c9376c46eb284f2b091f3eb98aec4fd64b03/references/detection/coco_utils.py#L63-L66)
        boxes = torch.as_tensor(boxes, dtype=torch.float32).reshape(-1, 4)
        boxes[:, 2:] += boxes[:, :2]
        boxes[:, 0::2].clamp_(min=0, max=width)
        boxes[:, 1::2].clamp_(min=0, max=height)

        # extract labels
        labels = torch.ones((num_fractures,), dtype=torch.int64)

        image_id = index
        area = box_area(boxes)

        # suppose all instances are not crowd
        iscrowd = torch.zeros((num_fractures,), dtype=torch.int64)

        # remove invalid boxes (https://github.com/pytorch/vision/blob/aa32c9376c46eb284f2b091f3eb98aec4fd64b03/references/detection/coco_utils.py#L82-L85)
        keep = (boxes[:, 3] > boxes[:, 1]) & (boxes[:, 2] > boxes[:, 0])
        boxes = boxes[keep]

        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd

        return target


In [14]:
from torchvision.transforms import v2 as T


def get_transform(train):
    transforms = []
    if train:
        transforms.append(T.RandomHorizontalFlip(0.5))
    transforms.append(T.ToImageTensor())
    transforms.append(T.ConvertImageDtype())
    # transforms.append(T.ToDtype(torch.float))
    # transforms.append(T.ToPureTensor())
    return T.Compose(transforms)


In [15]:
if not os.path.exists("lib/engine.py"):
    os.system(
        "wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/engine.py"
    )
if not os.path.exists("lib/utils.py"):
    os.system(
        "wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/utils.py"
    )
if not os.path.exists("lib/transforms.py"):
    os.system(
        "wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/transforms.py"
    )
if not os.path.exists("lib/coco_utils.py"):
    os.system(
        "wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/coco_utils.py"
    )
if not os.path.exists("lib/coco_eval.py"):
    os.system(
        "wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/coco_eval.py"
    )

# you also need to tweak them a bit to work with relative imports


In [16]:
from torchvision.models.detection import fasterrcnn_mobilenet_v3_large_fpn, FasterRCNN_MobileNet_V3_Large_FPN_Weights

model = fasterrcnn_mobilenet_v3_large_fpn(weights=FasterRCNN_MobileNet_V3_Large_FPN_Weights.DEFAULT)
dataset = FracAtlasDetection("../data/fracatlas", image_set="train", transform=get_transform(train=True))
data_loader = torch.utils.data.DataLoader(
  dataset,
  batch_size=2,
  shuffle=True,
  num_workers=4,
  collate_fn=lambda batch: tuple(zip(*batch)),
)

# For Training
images, targets = next(iter(data_loader))
images = list(image for image in images)
targets = [{k: v for k, v in t.items()} for t in targets]
output = model(images, targets)  # Returns losses and detections
print(output)

# For inference
model.eval()
x = [torch.rand(3, 300, 400), torch.rand(3, 500, 400)]
predictions = model(x)  # Returns predictions
print(predictions[0])


ModuleNotFoundError: No module named 'models'

In [ ]:
from lib.engine import train_one_epoch, evaluate

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print("device:", device)

# use our dataset and defined transformations
dataset = FracAtlasDetection('../data/fracatlas/', image_set="train", transform=get_transform(train=True))
dataset_test = FracAtlasDetection('../data/fracatlas/', image_set="test", transform=get_transform(train=False))

# split the dataset in train and test set
# indices = torch.randperm(len(dataset)).tolist()
# dataset = torch.utils.data.Subset(dataset, indices[:-50])
# dataset_test = torch.utils.data.Subset(dataset_test, indices[-50:])

# define training and validation data loaders
data_loader = torch.utils.data.DataLoader(
    dataset,
    batch_size=2,
    shuffle=True,
    num_workers=4,
    collate_fn=lambda batch: tuple(zip(*batch)),
)

data_loader_test = torch.utils.data.DataLoader(
    dataset_test,
    batch_size=1,
    shuffle=False,
    num_workers=4,
    collate_fn=lambda batch: tuple(zip(*batch)),
)

model = fasterrcnn_mobilenet_v3_large_fpn(weights=FasterRCNN_MobileNet_V3_Large_FPN_Weights.DEFAULT)

# move model to the right device
model.to(device)

# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(
    params,
    lr=0.005,
    momentum=0.9,
    weight_decay=0.0005
)

# and a learning rate scheduler
lr_scheduler = torch.optim.lr_scheduler.StepLR(
    optimizer,
    step_size=3,
    gamma=0.1
)

# let's train it for 5 epochs
num_epochs = 5

for epoch in range(num_epochs):
    # train for one epoch, printing every 10 iterations
    train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)
    # update the learning rate
    lr_scheduler.step()
    # evaluate on the test dataset
    evaluate(model, data_loader_test, device=device).analyze()

print("That's it!")

print("Saving")
torch.save(model, 'fracatlas_fasterrcnn-mobilenetv3_v1.pth')
print("Saved")


Epoch: [0]  [  0/287]  eta: 0:26:54  lr: 0.000022  loss: 0.9931 (0.9931)  loss_classifier: 0.4509 (0.4509)  loss_box_reg: 0.0344 (0.0344)  loss_objectness: 0.4696 (0.4696)  loss_rpn_box_reg: 0.0382 (0.0382)  time: 5.6271  data: 0.9940
Epoch: [0]  [ 10/287]  eta: 0:12:09  lr: 0.000197  loss: 1.0184 (1.1120)  loss_classifier: 0.4114 (0.4267)  loss_box_reg: 0.1109 (0.1560)  loss_objectness: 0.4258 (0.4726)  loss_rpn_box_reg: 0.0508 (0.0566)  time: 2.6335  data: 0.0952
Epoch: [0]  [ 20/287]  eta: 0:11:49  lr: 0.000372  loss: 1.0180 (1.0295)  loss_classifier: 0.3586 (0.3655)  loss_box_reg: 0.1960 (0.2201)  loss_objectness: 0.3674 (0.3872)  loss_rpn_box_reg: 0.0490 (0.0567)  time: 2.5077  data: 0.0084
Epoch: [0]  [ 30/287]  eta: 0:12:55  lr: 0.000546  loss: 0.6089 (0.8967)  loss_classifier: 0.2128 (0.3055)  loss_box_reg: 0.1998 (0.2070)  loss_objectness: 0.1902 (0.3344)  loss_rpn_box_reg: 0.0307 (0.0498)  time: 3.2304  data: 0.0116
Epoch: [0]  [ 40/287]  eta: 0:13:16  lr: 0.000721  loss: 0.5